In [26]:
from openai import OpenAI
import json
import os
import requests
import argparse

# Establishing connection to OpenAI
openAI_client = OpenAI() # defaults to os.environ.get("OPENAI_API_KEY")
openAI_model = "gpt-4o-mini"

# Establishing connection to you.com search API
you_API_key = os.environ.get("YOU_API_KEY")

def get_ai_web_snippets_for_query(query):
    """ 
    This function uses the you.com search API to get the top 10 search results for a given query.
    The results are returned as a list of dictionaries, where each dictionary contains the title, snippet, and URL of a search result.
    """
    headers = {"X-API-Key": you_API_key}
    params = {"num_web_results":"10","query": query}
    web_snippet_json = requests.get(
        f"https://api.ydc-index.io/search?query={query}",
        params=params,
        headers=headers,
    ).json()
    web_snippet = json.dumps(web_snippet_json)
    return web_snippet

def get_response_OpenAI(prompt):
    """
    This function uses the OpenAI API to generate a response to a given prompt.
    The prompt is a list of dictionaries, where each dictionary contains the role and content of a message.
    The response is the content of the message generated by the OpenAI model.
    """
    completion = openAI_client.chat.completions.create(
        model=openAI_model,
        messages=prompt,
        temperature=0,
    )
    return completion.choices[0].message.content

def classify_question(question: str):
    """
    This function uses the OpenAI API to classify a given question into one of 23 medical categories.
    The question is classified based on the content of the question.
    The category is returned as a string.
    """
    system_message = """
    You are a helpful medical assistant. Classify the provided question into one of the 23 categories listed below. Only return the category name.

    23 Categories:

    Association
    Cause
    Complication
    Contraindication
    Diagnosis
    Dosage
    Genetic changes
    Indication
    Information
    Ingredient
    Inheritance
    Interaction
    Organization
    Prevention
    Prognosis
    Resources
    Side effects
    Storage and disposal
    Susceptibility
    Symptom
    Tapering
    Treatment
    Usage
    """
    message_text = [{"role":"system","content":system_message},
                    {"role":"user","content":question}]
    category_of_question = get_response_OpenAI(prompt=message_text)

    return category_of_question

def get_question_focus_to_search_web(question: str):
    system_message = """
    You are a helpful medical assistant. Identify the focus of the question and provide a search query that can be used to find more information in the internet. Only return the search query.

    Example: "What is the treatment for diabetes?" -> "treatment for diabetes"
    """
    message_text = [{"role":"system","content":system_message},
                    {"role":"user","content":question}]
    search_query = get_response_OpenAI(prompt=message_text)

    return search_query

def get_answer_question_with_web_snippets(question: str, web_snippets: str):
    """
    This function generates an answer to a given question using the OpenAI API and the web snippets provided.
    The answer is generated based on the question and the content of the web snippets.
    The answer is returned as a string and contains the relevant URLs from the web snippets.
    """
    base_system_message = """
    You are a helpful medical assistant. Generate an answer to the provided question using your knowledge plus the web snippets provided below.
    Make sure the answer is relevant to the question and is supported by the web snippets. The web snippets are in JSON format.
    Only return the answer and useful supportive URLs.

    ### WEB SNIPPETS ###
    """
    system_message = base_system_message + "\n\n" + web_snippets
    message_text = [{"role":"system","content":system_message},
                    {"role":"user","content":question}]
    answer = get_response_OpenAI(prompt=message_text)

    return answer

def answer_question(question: str):
    """
    This function takes a medical question as input and returns an answer to the question.
    The answer is generated using the OpenAI API and the you.com websearch API.
    It also classifies the question into one of 23 medical categories and identifies the focus of the question to search the web.
    """
    # Classify the question into one of the 23 medical categories
    category = classify_question(question)
    
    # Identify the focus of the question to search the web
    search_query = get_question_focus_to_search_web(question)
    
    # Get web snippets for the search query
    web_snippets = get_ai_web_snippets_for_query(search_query)
    
    # Generate an answer using the OpenAI API and the web snippets
    answer = get_answer_question_with_web_snippets(question, web_snippets)
    
    complete_answer = f"Category: {category}\nSearch Query: {search_query}\nAnswer: {answer}"
    return category, search_query, answer, complete_answer

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Medical Question Assistant")
    parser.add_argument("question", type=str, help="The medical question to answer")
    args = parser.parse_args()

    category, search_query, answer, complete_answer = answer_question(args.question)
    print(complete_answer)


In [31]:
question = "Can you please send me as much information as possible on &quot;hypothyroidism&quot;. I was recently diagnosed with the disease and I am struggling to figure out what it is and how I got it. My name and address is: [NAME] [LOCATION] [CONTACT] Thank you in advance for your help!!!"
category, search_query, answer = answer_question(question)

Category of question: Information
Search query: "hypothyroidism information causes symptoms treatment"
Answer: Hypothyroidism is a condition where the thyroid gland does not produce enough thyroid hormones, which are crucial for regulating metabolism, energy levels, and overall bodily functions. Here’s a comprehensive overview:

### What is Hypothyroidism?
- **Definition**: Hypothyroidism occurs when the thyroid gland is underactive, leading to insufficient hormone production.
- **Hormones Involved**: The primary hormones affected are thyroxine (T4) and triiodothyronine (T3).

### Symptoms
Common symptoms of hypothyroidism include:
- Fatigue and weakness
- Weight gain
- Cold intolerance
- Dry skin and hair
- Constipation
- Depression
- Memory problems
- Muscle weakness

### Causes
Hypothyroidism can be caused by several factors, including:
- **Autoimmune Disease**: Hashimoto's thyroiditis is the most common cause, where the immune system attacks the thyroid.
- **Thyroid Surgery**: Remo

In [30]:
answer

"Pyridoxine (Vitamin B6) is sometimes used during pregnancy, particularly to help with nausea and vomiting. However, it is classified as FDA Pregnancy Category C, which means that risk cannot be ruled out. While there are no controlled studies in human pregnancy, it is noted that pyridoxine requirements may increase during pregnancy, and deficiency is common among pregnant women not on supplementation.\n\nIt's important to consult with your healthcare provider before using pyridoxine tablets during pregnancy to ensure it is appropriate for your specific situation.\n\nFor more information, you can visit:\n- [Pyridoxine Use During Pregnancy | Drugs.com](https://www.drugs.com/pregnancy/pyridoxine.html)"